# Grid Search dos parâmetros Batch_size e Epochs
Fonte: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

!pip install scikeras

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten,Dropout, LSTM, Embedding, Input
from tensorflow import keras
from keras import layers
from tensorflow.keras.optimizers import Adam, RMSprop

import pandas as pd
import tensorflow as tf
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from scikeras.wrappers import KerasClassifier
#optimizer = Adam(learning_rate=0.001)

def create_model():
  model=Sequential()
  model.add(LSTM(50 ,activation = 'relu',input_shape=(lags,1)))
  model.add(Dense(1, activation='sigmoid'))
  ## tamanho do lote e o número de épocas / 
  model.compile(optimizer= 'adam', loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [3]:
data = pd.read_csv('Inteligencia_Artificial/Trabalho de Conclusão de Curso/Base de Dados/2021/BBDC4_30m_2021.csv') 
data.head()

,time,open,high,low,close,Volume,Volume MA
0,2021-01-04T10:00:00-03:00,21.814317,21.814317,21.505287,21.521135,1880012,1384668.55
1,2021-01-04T10:30:00-03:00,21.529059,21.576602,21.441896,21.497363,1532703,1368010.00
2,2021-01-04T11:00:00-03:00,21.505287,21.552830,21.473592,21.481516,877715,1345874.15
3,2021-01-04T11:30:00-03:00,21.497363,21.521135,21.132867,21.188334,5465393,1548509.50
4,2021-01-04T12:00:00-03:00,21.188334,21.227953,21.085324,21.124943,4150374,1694177.35


In [4]:
seed = 7
tf.random.set_seed(seed)

In [5]:
symbol = "close"
data = pd.DataFrame(data[symbol])
data.head()
data['return'] = np.log(data['close']/data['close'].shift(1))
temp = pd.DataFrame({})
temp['direction'] = np.where(data['return'] > 0, 1, 0)
#display(temp)

lags = 4

cols = []
for lag in range(1, lags + 1):
     col = f'lag_{lag}'
     temp[col] = data['return'].shift(lag)
     cols.append(col)
temp.dropna(inplace=True)

#display(temp)

temp.index = range(len(temp['direction']))

np.set_printoptions(threshold=False)

def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  Y = df_as_np[:,0]

  middle_matrix = df_as_np[:,1:5]

  X = middle_matrix.reshape((len(Y), middle_matrix.shape[1], 1))

  return X, Y.astype(int)

X, y = windowed_df_to_date_X_y(temp)

#-----------------------------------------------------------
# Define os conjuntos de dados de treinamento, teste e validação
#-----------------------------------------------------------
q_80 = int(len(X) * .8)

X_train, y_train = X[:q_80], y[:q_80]
X_test, y_test = X[q_80:], y[q_80:]

mu, std = X_train.mean(), X_train.std()
X_train = (X_train - mu) / std

X_test = (X_test - mu) / std

print(X_train)
print(X_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

[[[-0.59602594]
  [-2.76167063]
  [-0.14079378]
  [-0.2149112 ]]

 [[-1.5097588 ]
  [-0.59602594]
  [-2.76167063]
  [-0.14079378]]

 [[-0.37336195]
  [-1.5097588 ]
  [-0.59602594]
  [-2.76167063]]

 ...

 [[ 1.05426085]
  [-0.94370054]
  [ 0.863996  ]
  [-0.84834892]]

 [[ 0.85997355]
  [ 1.05426085]
  [-0.94370054]
  [ 0.863996  ]]

 [[-1.12918969]
  [ 0.85997355]
  [ 1.05426085]
  [-0.94370054]]]
[[[-1.04008709]
  [-1.12918969]
  [ 0.85997355]
  [ 1.05426085]]

 [[-0.66188858]
  [-1.04008709]
  [-1.12918969]
  [ 0.85997355]]

 [[ 1.53532774]
  [-0.66188858]
  [-1.04008709]
  [-1.12918969]]

 ...

 [[-0.51496261]
  [ 0.53060969]
  [ 1.26805957]
  [-0.30797794]]

 [[ 0.00782354]
  [-0.51496261]
  [ 0.53060969]
  [ 1.26805957]]

 [[-0.20167122]
  [ 0.00782354]
  [-0.51496261]
  [ 0.53060969]]]


((3146, 4, 1), (3146,), (787, 4, 1), (787,))

In [6]:
# Quem usa TimeSeriesSplit?
# https://github.com/dduemig/Stanford-Project-Predicting-stock-prices-using-a-LSTM-Network/blob/master/Final%20Project.ipynb# https://towardsdatascience.com/cross-validation-and-grid-search-efa64b127c1b (CV parameter)
# Arthur (doutorado)

#https://scikit-learn.org/stable/modules/cross_validation.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
from sklearn.model_selection import TimeSeriesSplit

dev_size=0.2 
n_splits=int((1//dev_size)-1)   # using // for integer division
tscv=TimeSeriesSplit(n_splits=n_splits) 
tscv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)

In [7]:
# create model
# tamanho do lote e o número de épocas / 
model = KerasClassifier(model=create_model, verbose=0)


In [ ]:
for i in range(30):
  # define the grid search parameters
  batch_size = [10, 20, 40, 60, 80, 100]
  epochs = [10, 50, 100, 500]
  param_grid = dict(batch_size=batch_size, epochs=epochs)
  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=tscv)
  grid_result = grid.fit(X_train, y_train)

  # summarize results

  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
## 1º Configuração

Best: 0.537580 using {'batch_size': 40, 'epochs': 500}
Best: 0.517622 using {'batch_size': 60, 'epochs': 50}
Best: 0.525265 using {'batch_size': 60, 'epochs': 500}
Best: 0.529512 using {'batch_size': 10, 'epochs': 500}
Best: 0.526115 using {'batch_size': 40, 'epochs': 500}
Best: 0.529936 using {'batch_size': 40, 'epochs': 500}
Best: 0.524416 using {'batch_size': 60, 'epochs': 500}
Best: 0.530361 using {'batch_size': 100, 'epochs': 500}
Best: 0.524841 using {'batch_size': 20, 'epochs': 500}
Best: 0.529512 using {'batch_size': 20, 'epochs': 500}
Best: 0.531210 using {'batch_size': 20, 'epochs': 500}
Best: 0.528238 using {'batch_size': 40, 'epochs': 500}
Best: 0.527389 using {'batch_size': 10, 'epochs': 500}
Best: 0.529087 using {'batch_size': 20, 'epochs': 500}
Best: 0.530361 using {'batch_size': 20, 'epochs': 500}
Best: 0.526539 using {'batch_size': 10, 'epochs': 500}
Best: 0.534607 using {'batch_size': 10, 'epochs': 500}
Best: 0.529936 using {'batch_size': 20, 'epochs': 500}
Best: 0.528238 using {'batch_size': 10, 'epochs': 500}
Best: 0.531635 using {'batch_size': 10, 'epochs': 500}
Best: 0.521019 using {'batch_size': 60, 'epochs': 500}
Best: 0.525265 using {'batch_size': 10, 'epochs': 500}
Best: 0.530786 using {'batch_size': 80, 'epochs': 500}
Best: 0.526539 using {'batch_size': 60, 'epochs': 500}
Best: 0.529936 using {'batch_size': 20, 'epochs': 500}
---> Best: 0.525265 using {'batch_size': 10, 'epochs': 500}
Best: 0.528662 using {'batch_size': 20, 'epochs': 500}
Best: 0.526539 using {'batch_size': 10, 'epochs': 500}
Best: 0.520594 using {'batch_size': 60, 'epochs': 500}
Best: 0.531210 using {'batch_size': 40, 'epochs': 500}

## 2º Configuração

In [ ]:
#optimizer = Adam(learning_rate=0.001)

def create_model():
  model=Sequential()
  model.add(LSTM(50 ,activation = 'relu',input_shape=(lags,1)))
  model.add(Dense(1, activation='sigmoid'))
  ## tamanho do lote e o número de épocas / 
  model.compile(optimizer= 'adam', loss='binary_crossentropy',metrics=['accuracy'])
  return model


Best: 0.543681 using {'batch_size': 40, 'epochs': 10}
0.542833 (0.010405) with: {'batch_size': 10, 'epochs': 10}
0.539864 (0.006262) with: {'batch_size': 10, 'epochs': 50}
0.537320 (0.016632) with: {'batch_size': 10, 'epochs': 100}
0.490670 (0.013794) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 20, 'epochs': 10}
0.534775 (0.000600) with: {'batch_size': 20, 'epochs': 50}
0.543681 (0.012905) with: {'batch_size': 20, 'epochs': 100}
0.509754 (0.009652) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.537320 (0.000600) with: {'batch_size': 40, 'epochs': 50}
0.537744 (0.013397) with: {'batch_size': 40, 'epochs': 100}
0.513571 (0.004198) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535623 (0.007272) with: {'batch_size': 60, 'epochs': 50}
0.536047 (0.012566) with: {'batch_size': 60, 'epochs': 100}
0.532231 (0.010457) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.536047 (0.003174) with: {'batch_size': 80, 'epochs': 50}
0.533927 (0.017882) with: {'batch_size': 80, 'epochs': 100}
0.520356 (0.011980) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.536047 (0.002399) with: {'batch_size': 100, 'epochs': 50}
0.536047 (0.010812) with: {'batch_size': 100, 'epochs': 100}
0.520780 (0.016138) with: {'batch_size': 100, 'epochs': 500}
Best: 0.545802 using {'batch_size': 40, 'epochs': 100}
0.542409 (0.009076) with: {'batch_size': 10, 'epochs': 10}
0.537744 (0.007296) with: {'batch_size': 10, 'epochs': 50}
0.533503 (0.008397) with: {'batch_size': 10, 'epochs': 100}
0.506361 (0.015302) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 20, 'epochs': 10}
0.538592 (0.003933) with: {'batch_size': 20, 'epochs': 50}
0.541561 (0.006087) with: {'batch_size': 20, 'epochs': 100}
0.516115 (0.012306) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 40, 'epochs': 10}
0.533503 (0.003339) with: {'batch_size': 40, 'epochs': 50}
0.545802 (0.018785) with: {'batch_size': 40, 'epochs': 100}
0.518236 (0.006759) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535199 (0.002162) with: {'batch_size': 60, 'epochs': 50}
0.536896 (0.013504) with: {'batch_size': 60, 'epochs': 100}
0.522053 (0.014593) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 80, 'epochs': 10}
0.534351 (0.004528) with: {'batch_size': 80, 'epochs': 50}
0.534351 (0.013624) with: {'batch_size': 80, 'epochs': 100}
0.518660 (0.015030) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.537320 (0.005331) with: {'batch_size': 100, 'epochs': 50}
0.538592 (0.008835) with: {'batch_size': 100, 'epochs': 100}
0.538592 (0.007657) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 100, 'epochs': 10}
0.541985 (0.010543) with: {'batch_size': 10, 'epochs': 10}
0.536047 (0.007797) with: {'batch_size': 10, 'epochs': 50}
0.532231 (0.005998) with: {'batch_size': 10, 'epochs': 100}
0.499152 (0.021221) with: {'batch_size': 10, 'epochs': 500}
0.542833 (0.008956) with: {'batch_size': 20, 'epochs': 10}
0.533503 (0.003648) with: {'batch_size': 20, 'epochs': 50}
0.521628 (0.014728) with: {'batch_size': 20, 'epochs': 100}
0.499576 (0.010457) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.539864 (0.007657) with: {'batch_size': 40, 'epochs': 50}
0.533503 (0.008332) with: {'batch_size': 40, 'epochs': 100}
0.529686 (0.006917) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536047 (0.001587) with: {'batch_size': 60, 'epochs': 50}
0.543681 (0.008712) with: {'batch_size': 60, 'epochs': 100}
0.527142 (0.034286) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.532231 (0.005331) with: {'batch_size': 80, 'epochs': 50}
0.537320 (0.002614) with: {'batch_size': 80, 'epochs': 100}
0.521204 (0.016825) with: {'batch_size': 80, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 100, 'epochs': 10}
0.536472 (0.002162) with: {'batch_size': 100, 'epochs': 50}
0.538592 (0.008397) with: {'batch_size': 100, 'epochs': 100}
0.520780 (0.021094) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 40, 'epochs': 10}
0.541561 (0.006759) with: {'batch_size': 10, 'epochs': 10}
0.532655 (0.004325) with: {'batch_size': 10, 'epochs': 50}
0.540288 (0.019304) with: {'batch_size': 10, 'epochs': 100}
0.503393 (0.017761) with: {'batch_size': 10, 'epochs': 500}
0.542409 (0.009076) with: {'batch_size': 20, 'epochs': 10}
0.527142 (0.011204) with: {'batch_size': 20, 'epochs': 50}
0.542833 (0.009652) with: {'batch_size': 20, 'epochs': 100}
0.515691 (0.012821) with: {'batch_size': 20, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 40, 'epochs': 10}
0.527142 (0.002162) with: {'batch_size': 40, 'epochs': 50}
0.540712 (0.011707) with: {'batch_size': 40, 'epochs': 100}
0.513571 (0.000600) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536047 (0.003339) with: {'batch_size': 60, 'epochs': 50}
0.529686 (0.016238) with: {'batch_size': 60, 'epochs': 100}
0.535199 (0.016171) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 80, 'epochs': 10}
0.536472 (0.001587) with: {'batch_size': 80, 'epochs': 50}
0.534351 (0.001039) with: {'batch_size': 80, 'epochs': 100}
0.520780 (0.010661) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.538592 (0.001587) with: {'batch_size': 100, 'epochs': 50}
0.533927 (0.006347) with: {'batch_size': 100, 'epochs': 100}
0.528414 (0.015030) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544953 using {'batch_size': 20, 'epochs': 10}
0.543257 (0.010994) with: {'batch_size': 10, 'epochs': 10}
0.539440 (0.005194) with: {'batch_size': 10, 'epochs': 50}
0.539864 (0.011814) with: {'batch_size': 10, 'epochs': 100}
0.507209 (0.020497) with: {'batch_size': 10, 'epochs': 500}
0.544953 (0.011108) with: {'batch_size': 20, 'epochs': 10}
0.533079 (0.009056) with: {'batch_size': 20, 'epochs': 50}
0.535623 (0.012466) with: {'batch_size': 20, 'epochs': 100}
0.509754 (0.009652) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.533927 (0.006087) with: {'batch_size': 40, 'epochs': 50}
0.543257 (0.011236) with: {'batch_size': 40, 'epochs': 100}
0.510602 (0.014556) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535199 (0.003648) with: {'batch_size': 60, 'epochs': 50}
0.531807 (0.010994) with: {'batch_size': 60, 'epochs': 100}
0.519084 (0.017351) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 80, 'epochs': 10}
0.536047 (0.005721) with: {'batch_size': 80, 'epochs': 50}
0.541137 (0.006262) with: {'batch_size': 80, 'epochs': 100}
0.535623 (0.011707) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.543257 (0.002748) with: {'batch_size': 100, 'epochs': 50}
0.536896 (0.002078) with: {'batch_size': 100, 'epochs': 100}
0.523749 (0.011395) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544953 using {'batch_size': 40, 'epochs': 100}
0.543681 (0.010812) with: {'batch_size': 10, 'epochs': 10}
0.541561 (0.010089) with: {'batch_size': 10, 'epochs': 50}
0.533503 (0.014180) with: {'batch_size': 10, 'epochs': 100}
0.505937 (0.022576) with: {'batch_size': 10, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 20, 'epochs': 10}
0.534775 (0.002162) with: {'batch_size': 20, 'epochs': 50}
0.536896 (0.010796) with: {'batch_size': 20, 'epochs': 100}
0.516539 (0.001039) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.536896 (0.002078) with: {'batch_size': 40, 'epochs': 50}
0.544953 (0.007370) with: {'batch_size': 40, 'epochs': 100}
0.508058 (0.009652) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.533927 (0.001200) with: {'batch_size': 60, 'epochs': 50}
0.532231 (0.009076) with: {'batch_size': 60, 'epochs': 100}
0.513147 (0.011443) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 80, 'epochs': 10}
0.537320 (0.001587) with: {'batch_size': 80, 'epochs': 50}
0.534775 (0.006087) with: {'batch_size': 80, 'epochs': 100}
0.531807 (0.015089) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.533503 (0.000600) with: {'batch_size': 100, 'epochs': 50}
0.535623 (0.007843) with: {'batch_size': 100, 'epochs': 100}
0.526718 (0.007491) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 20, 'epochs': 50}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.532231 (0.017670) with: {'batch_size': 10, 'epochs': 50}
0.525869 (0.016104) with: {'batch_size': 10, 'epochs': 100}
0.502969 (0.016436) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.544105 (0.001587) with: {'batch_size': 20, 'epochs': 50}
0.527990 (0.002748) with: {'batch_size': 20, 'epochs': 100}
0.516115 (0.011676) with: {'batch_size': 20, 'epochs': 500}
0.544105 (0.010661) with: {'batch_size': 40, 'epochs': 10}
0.536896 (0.003745) with: {'batch_size': 40, 'epochs': 50}
0.533927 (0.004684) with: {'batch_size': 40, 'epochs': 100}
0.518236 (0.001587) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536047 (0.003339) with: {'batch_size': 60, 'epochs': 50}
0.537320 (0.015868) with: {'batch_size': 60, 'epochs': 100}
0.525869 (0.013518) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.535199 (0.005331) with: {'batch_size': 80, 'epochs': 50}
0.537744 (0.013397) with: {'batch_size': 80, 'epochs': 100}
0.527566 (0.011395) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.535199 (0.005331) with: {'batch_size': 100, 'epochs': 50}
0.539864 (0.005721) with: {'batch_size': 100, 'epochs': 100}
0.524597 (0.021848) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 60, 'epochs': 10}
0.541137 (0.007370) with: {'batch_size': 10, 'epochs': 10}
0.532231 (0.011676) with: {'batch_size': 10, 'epochs': 50}
0.530534 (0.007272) with: {'batch_size': 10, 'epochs': 100}
0.508482 (0.005331) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 20, 'epochs': 10}
0.533503 (0.002399) with: {'batch_size': 20, 'epochs': 50}
0.541561 (0.008332) with: {'batch_size': 20, 'epochs': 100}
0.514419 (0.003174) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.534775 (0.006597) with: {'batch_size': 40, 'epochs': 50}
0.527566 (0.003933) with: {'batch_size': 40, 'epochs': 100}
0.510602 (0.008956) with: {'batch_size': 40, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 60, 'epochs': 10}
0.538592 (0.001587) with: {'batch_size': 60, 'epochs': 50}
0.536472 (0.012174) with: {'batch_size': 60, 'epochs': 100}
0.520356 (0.022473) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.536896 (0.008503) with: {'batch_size': 80, 'epochs': 50}
0.534351 (0.014654) with: {'batch_size': 80, 'epochs': 100}
0.525869 (0.013950) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.539016 (0.006597) with: {'batch_size': 100, 'epochs': 50}
0.532655 (0.006347) with: {'batch_size': 100, 'epochs': 100}
0.532231 (0.012988) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 60, 'epochs': 10}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.537320 (0.010457) with: {'batch_size': 10, 'epochs': 50}
0.541561 (0.007071) with: {'batch_size': 10, 'epochs': 100}
0.504241 (0.009652) with: {'batch_size': 10, 'epochs': 500}
0.540712 (0.006812) with: {'batch_size': 20, 'epochs': 10}
0.541561 (0.008956) with: {'batch_size': 20, 'epochs': 50}
0.535623 (0.011707) with: {'batch_size': 20, 'epochs': 100}
0.527142 (0.005124) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 40, 'epochs': 10}
0.533927 (0.003339) with: {'batch_size': 40, 'epochs': 50}
0.536047 (0.004909) with: {'batch_size': 40, 'epochs': 100}
0.511026 (0.008956) with: {'batch_size': 40, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 60, 'epochs': 10}
0.536896 (0.003116) with: {'batch_size': 60, 'epochs': 50}
0.540712 (0.007491) with: {'batch_size': 60, 'epochs': 100}
0.521204 (0.021897) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.537320 (0.001587) with: {'batch_size': 80, 'epochs': 50}
0.537744 (0.009652) with: {'batch_size': 80, 'epochs': 100}
0.528414 (0.007370) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.540288 (0.005907) with: {'batch_size': 100, 'epochs': 50}
0.532231 (0.008712) with: {'batch_size': 100, 'epochs': 100}
0.521204 (0.020991) with: {'batch_size': 100, 'epochs': 500}
Best: 0.543681 using {'batch_size': 10, 'epochs': 10}
0.543681 (0.010812) with: {'batch_size': 10, 'epochs': 10}
0.527990 (0.007491) with: {'batch_size': 10, 'epochs': 50}
0.533927 (0.011583) with: {'batch_size': 10, 'epochs': 100}
0.508906 (0.014053) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.533503 (0.005721) with: {'batch_size': 20, 'epochs': 50}
0.538592 (0.018852) with: {'batch_size': 20, 'epochs': 100}
0.513147 (0.003933) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 40, 'epochs': 10}
0.539016 (0.004198) with: {'batch_size': 40, 'epochs': 50}
0.528414 (0.010405) with: {'batch_size': 40, 'epochs': 100}
0.505513 (0.005907) with: {'batch_size': 40, 'epochs': 500}
0.541985 (0.010845) with: {'batch_size': 60, 'epochs': 10}
0.532231 (0.001200) with: {'batch_size': 60, 'epochs': 50}
0.536047 (0.004325) with: {'batch_size': 60, 'epochs': 100}
0.529686 (0.022288) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 80, 'epochs': 10}
0.533927 (0.002614) with: {'batch_size': 80, 'epochs': 50}
0.536896 (0.009233) with: {'batch_size': 80, 'epochs': 100}
0.516115 (0.020785) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.539440 (0.002078) with: {'batch_size': 100, 'epochs': 50}
0.540288 (0.008069) with: {'batch_size': 100, 'epochs': 100}
0.529686 (0.011490) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 10, 'epochs': 10}
0.544105 (0.010661) with: {'batch_size': 10, 'epochs': 10}
0.526718 (0.007491) with: {'batch_size': 10, 'epochs': 50}
0.530958 (0.016825) with: {'batch_size': 10, 'epochs': 100}
0.500424 (0.012306) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.537744 (0.005907) with: {'batch_size': 20, 'epochs': 50}
0.542833 (0.002614) with: {'batch_size': 20, 'epochs': 100}
0.508058 (0.009596) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.530110 (0.003648) with: {'batch_size': 40, 'epochs': 50}
0.529262 (0.011427) with: {'batch_size': 40, 'epochs': 100}
0.517388 (0.003933) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535623 (0.001039) with: {'batch_size': 60, 'epochs': 50}
0.533927 (0.003648) with: {'batch_size': 60, 'epochs': 100}
0.523325 (0.017912) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.538168 (0.001799) with: {'batch_size': 80, 'epochs': 50}
0.539440 (0.002748) with: {'batch_size': 80, 'epochs': 100}
0.530110 (0.016171) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.535623 (0.003745) with: {'batch_size': 100, 'epochs': 50}
0.541137 (0.006262) with: {'batch_size': 100, 'epochs': 100}
0.518660 (0.009076) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 40, 'epochs': 10}
0.541985 (0.008503) with: {'batch_size': 10, 'epochs': 10}
0.519932 (0.017143) with: {'batch_size': 10, 'epochs': 50}
0.532231 (0.010812) with: {'batch_size': 10, 'epochs': 100}
0.508058 (0.023467) with: {'batch_size': 10, 'epochs': 500}
0.542409 (0.009076) with: {'batch_size': 20, 'epochs': 10}
0.530958 (0.003339) with: {'batch_size': 20, 'epochs': 50}
0.538592 (0.009652) with: {'batch_size': 20, 'epochs': 100}
0.511874 (0.016501) with: {'batch_size': 20, 'epochs': 500}
0.544529 (0.011980) with: {'batch_size': 40, 'epochs': 10}
0.536047 (0.001587) with: {'batch_size': 40, 'epochs': 50}
0.540712 (0.006319) with: {'batch_size': 40, 'epochs': 100}
0.514419 (0.010249) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536047 (0.003933) with: {'batch_size': 60, 'epochs': 50}
0.538168 (0.014654) with: {'batch_size': 60, 'epochs': 100}
0.528838 (0.015559) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.533927 (0.003339) with: {'batch_size': 80, 'epochs': 50}
0.536896 (0.013504) with: {'batch_size': 80, 'epochs': 100}
0.527566 (0.011490) with: {'batch_size': 80, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 100, 'epochs': 10}
0.537744 (0.000600) with: {'batch_size': 100, 'epochs': 50}
0.539864 (0.005124) with: {'batch_size': 100, 'epochs': 100}
0.527990 (0.013742) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544953 using {'batch_size': 10, 'epochs': 100}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.538168 (0.006812) with: {'batch_size': 10, 'epochs': 50}
0.544953 (0.007657) with: {'batch_size': 10, 'epochs': 100}
0.504665 (0.008332) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.533503 (0.001587) with: {'batch_size': 20, 'epochs': 50}
0.535623 (0.003745) with: {'batch_size': 20, 'epochs': 100}
0.526718 (0.001039) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.539864 (0.002162) with: {'batch_size': 40, 'epochs': 50}
0.541137 (0.010611) with: {'batch_size': 40, 'epochs': 100}
0.517812 (0.018727) with: {'batch_size': 40, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 60, 'epochs': 10}
0.536896 (0.003599) with: {'batch_size': 60, 'epochs': 50}
0.536472 (0.006679) with: {'batch_size': 60, 'epochs': 100}
0.523749 (0.006917) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.538592 (0.003339) with: {'batch_size': 80, 'epochs': 50}
0.533927 (0.007515) with: {'batch_size': 80, 'epochs': 100}
0.527566 (0.024414) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.532231 (0.001587) with: {'batch_size': 100, 'epochs': 50}
0.539016 (0.006087) with: {'batch_size': 100, 'epochs': 100}
0.522053 (0.014556) with: {'batch_size': 100, 'epochs': 500}
Best: 0.543681 using {'batch_size': 80, 'epochs': 10}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.531807 (0.003116) with: {'batch_size': 10, 'epochs': 50}
0.530110 (0.010912) with: {'batch_size': 10, 'epochs': 100}
0.510178 (0.009521) with: {'batch_size': 10, 'epochs': 500}
0.541561 (0.007934) with: {'batch_size': 20, 'epochs': 10}
0.537320 (0.010508) with: {'batch_size': 20, 'epochs': 50}
0.540288 (0.004684) with: {'batch_size': 20, 'epochs': 100}
0.503817 (0.009521) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 40, 'epochs': 10}
0.541985 (0.004155) with: {'batch_size': 40, 'epochs': 50}
0.530534 (0.011427) with: {'batch_size': 40, 'epochs': 100}
0.519084 (0.021008) with: {'batch_size': 40, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 60, 'epochs': 10}
0.536896 (0.002078) with: {'batch_size': 60, 'epochs': 50}
0.532231 (0.009982) with: {'batch_size': 60, 'epochs': 100}
0.527990 (0.013504) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.534775 (0.002999) with: {'batch_size': 80, 'epochs': 50}
0.539016 (0.002999) with: {'batch_size': 80, 'epochs': 100}
0.511450 (0.021815) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.539016 (0.000600) with: {'batch_size': 100, 'epochs': 50}
0.537320 (0.005229) with: {'batch_size': 100, 'epochs': 100}
0.516115 (0.012694) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 40, 'epochs': 10}
0.543681 (0.010089) with: {'batch_size': 10, 'epochs': 10}
0.529686 (0.013518) with: {'batch_size': 10, 'epochs': 50}
0.530110 (0.012821) with: {'batch_size': 10, 'epochs': 100}
0.502969 (0.008956) with: {'batch_size': 10, 'epochs': 500}
0.542833 (0.008332) with: {'batch_size': 20, 'epochs': 10}
0.537320 (0.006759) with: {'batch_size': 20, 'epochs': 50}
0.538592 (0.013397) with: {'batch_size': 20, 'epochs': 100}
0.504665 (0.014556) with: {'batch_size': 20, 'epochs': 500}
0.544105 (0.012174) with: {'batch_size': 40, 'epochs': 10}
0.533927 (0.002614) with: {'batch_size': 40, 'epochs': 50}
0.532655 (0.008835) with: {'batch_size': 40, 'epochs': 100}
0.528414 (0.015420) with: {'batch_size': 40, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 60, 'epochs': 10}
0.537320 (0.007797) with: {'batch_size': 60, 'epochs': 50}
0.542833 (0.007797) with: {'batch_size': 60, 'epochs': 100}
0.534351 (0.015992) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.538168 (0.002748) with: {'batch_size': 80, 'epochs': 50}
0.534775 (0.007370) with: {'batch_size': 80, 'epochs': 100}
0.522477 (0.015766) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.536896 (0.002748) with: {'batch_size': 100, 'epochs': 50}
0.539016 (0.004798) with: {'batch_size': 100, 'epochs': 100}
0.524173 (0.015720) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 10, 'epochs': 10}
0.544529 (0.010543) with: {'batch_size': 10, 'epochs': 10}
0.534351 (0.016227) with: {'batch_size': 10, 'epochs': 50}
0.522901 (0.011707) with: {'batch_size': 10, 'epochs': 100}
0.505513 (0.018737) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.539016 (0.005229) with: {'batch_size': 20, 'epochs': 50}
0.540288 (0.008069) with: {'batch_size': 20, 'epochs': 100}
0.509330 (0.006087) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.011583) with: {'batch_size': 40, 'epochs': 10}
0.537320 (0.004198) with: {'batch_size': 40, 'epochs': 50}
0.543257 (0.005784) with: {'batch_size': 40, 'epochs': 100}
0.508058 (0.008069) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.533927 (0.002614) with: {'batch_size': 60, 'epochs': 50}
0.539864 (0.004198) with: {'batch_size': 60, 'epochs': 100}
0.511450 (0.028277) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.532231 (0.000600) with: {'batch_size': 80, 'epochs': 50}
0.533503 (0.006679) with: {'batch_size': 80, 'epochs': 100}
0.525021 (0.019304) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.538592 (0.006917) with: {'batch_size': 100, 'epochs': 50}
0.530110 (0.006679) with: {'batch_size': 100, 'epochs': 100}
0.525445 (0.017751) with: {'batch_size': 100, 'epochs': 500}
Best: 0.543681 using {'batch_size': 20, 'epochs': 10}
0.543257 (0.009521) with: {'batch_size': 10, 'epochs': 10}
0.526718 (0.009910) with: {'batch_size': 10, 'epochs': 50}
0.540712 (0.011236) with: {'batch_size': 10, 'epochs': 100}
0.513995 (0.021815) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.535199 (0.007071) with: {'batch_size': 20, 'epochs': 50}
0.538592 (0.005998) with: {'batch_size': 20, 'epochs': 100}
0.504241 (0.009311) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.535199 (0.005907) with: {'batch_size': 40, 'epochs': 50}
0.534775 (0.014593) with: {'batch_size': 40, 'epochs': 100}
0.517388 (0.030505) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.538168 (0.001799) with: {'batch_size': 60, 'epochs': 50}
0.531383 (0.013833) with: {'batch_size': 60, 'epochs': 100}
0.525869 (0.012988) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.539440 (0.005784) with: {'batch_size': 80, 'epochs': 50}
0.525445 (0.013584) with: {'batch_size': 80, 'epochs': 100}
0.519084 (0.017101) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.537320 (0.001200) with: {'batch_size': 100, 'epochs': 50}
0.538168 (0.005784) with: {'batch_size': 100, 'epochs': 100}
0.508058 (0.017080) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 10, 'epochs': 10}
0.544529 (0.010994) with: {'batch_size': 10, 'epochs': 10}
0.535623 (0.008996) with: {'batch_size': 10, 'epochs': 50}
0.532231 (0.007797) with: {'batch_size': 10, 'epochs': 100}
0.492366 (0.010388) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 20, 'epochs': 10}
0.537744 (0.005229) with: {'batch_size': 20, 'epochs': 50}
0.525445 (0.004760) with: {'batch_size': 20, 'epochs': 100}
0.506361 (0.019462) with: {'batch_size': 20, 'epochs': 500}
0.542833 (0.010405) with: {'batch_size': 40, 'epochs': 10}
0.536472 (0.006347) with: {'batch_size': 40, 'epochs': 50}
0.541561 (0.008069) with: {'batch_size': 40, 'epochs': 100}
0.508906 (0.014394) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535199 (0.005331) with: {'batch_size': 60, 'epochs': 50}
0.536896 (0.013016) with: {'batch_size': 60, 'epochs': 100}
0.525445 (0.019819) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 80, 'epochs': 10}
0.536047 (0.002999) with: {'batch_size': 80, 'epochs': 50}
0.538168 (0.005497) with: {'batch_size': 80, 'epochs': 100}
0.517812 (0.014281) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.538592 (0.000600) with: {'batch_size': 100, 'epochs': 50}
0.531383 (0.019051) with: {'batch_size': 100, 'epochs': 100}
0.531383 (0.016696) with: {'batch_size': 100, 'epochs': 500}
Best: 0.546650 using {'batch_size': 80, 'epochs': 100}
0.541137 (0.008397) with: {'batch_size': 10, 'epochs': 10}
0.539016 (0.007296) with: {'batch_size': 10, 'epochs': 50}
0.533079 (0.012975) with: {'batch_size': 10, 'epochs': 100}
0.501696 (0.011443) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.530534 (0.004528) with: {'batch_size': 20, 'epochs': 50}
0.536896 (0.013624) with: {'batch_size': 20, 'epochs': 100}
0.497880 (0.013357) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.534351 (0.001039) with: {'batch_size': 40, 'epochs': 50}
0.533503 (0.007866) with: {'batch_size': 40, 'epochs': 100}
0.509330 (0.006262) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.539016 (0.005721) with: {'batch_size': 60, 'epochs': 50}
0.534775 (0.009076) with: {'batch_size': 60, 'epochs': 100}
0.515691 (0.017578) with: {'batch_size': 60, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 80, 'epochs': 10}
0.539016 (0.002162) with: {'batch_size': 80, 'epochs': 50}
0.546650 (0.005721) with: {'batch_size': 80, 'epochs': 100}
0.519508 (0.013357) with: {'batch_size': 80, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 100, 'epochs': 10}
0.539440 (0.004155) with: {'batch_size': 100, 'epochs': 50}
0.536896 (0.007491) with: {'batch_size': 100, 'epochs': 100}
0.515267 (0.018260) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 10, 'epochs': 50}
0.542409 (0.010089) with: {'batch_size': 10, 'epochs': 10}
0.544529 (0.007272) with: {'batch_size': 10, 'epochs': 50}
0.525869 (0.008069) with: {'batch_size': 10, 'epochs': 100}
0.513571 (0.013235) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.011236) with: {'batch_size': 20, 'epochs': 10}
0.531807 (0.008245) with: {'batch_size': 20, 'epochs': 50}
0.533927 (0.014740) with: {'batch_size': 20, 'epochs': 100}
0.515267 (0.001799) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 40, 'epochs': 10}
0.537744 (0.006759) with: {'batch_size': 40, 'epochs': 50}
0.534351 (0.009910) with: {'batch_size': 40, 'epochs': 100}
0.509754 (0.011108) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.531383 (0.006917) with: {'batch_size': 60, 'epochs': 50}
0.535199 (0.007515) with: {'batch_size': 60, 'epochs': 100}
0.525869 (0.015208) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.536472 (0.003933) with: {'batch_size': 80, 'epochs': 50}
0.532231 (0.013235) with: {'batch_size': 80, 'epochs': 100}
0.533079 (0.012638) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.539016 (0.003174) with: {'batch_size': 100, 'epochs': 50}
0.534351 (0.003745) with: {'batch_size': 100, 'epochs': 100}
0.534351 (0.018899) with: {'batch_size': 100, 'epochs': 500}
Best: 0.549618 using {'batch_size': 40, 'epochs': 100}
0.544105 (0.011395) with: {'batch_size': 10, 'epochs': 10}
0.539864 (0.005721) with: {'batch_size': 10, 'epochs': 50}
0.527142 (0.009652) with: {'batch_size': 10, 'epochs': 100}
0.506785 (0.014407) with: {'batch_size': 10, 'epochs': 500}
0.544105 (0.010661) with: {'batch_size': 20, 'epochs': 10}
0.542833 (0.006759) with: {'batch_size': 20, 'epochs': 50}
0.544105 (0.006917) with: {'batch_size': 20, 'epochs': 100}
0.509330 (0.008461) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.539864 (0.004684) with: {'batch_size': 40, 'epochs': 50}
0.549618 (0.006812) with: {'batch_size': 40, 'epochs': 100}
0.515267 (0.007197) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.539016 (0.002614) with: {'batch_size': 60, 'epochs': 50}
0.530958 (0.007934) with: {'batch_size': 60, 'epochs': 100}
0.511874 (0.016436) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.531807 (0.001039) with: {'batch_size': 80, 'epochs': 50}
0.536472 (0.006679) with: {'batch_size': 80, 'epochs': 100}
0.526293 (0.011395) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.536472 (0.003933) with: {'batch_size': 100, 'epochs': 50}
0.540288 (0.001587) with: {'batch_size': 100, 'epochs': 100}
0.523325 (0.006917) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 20, 'epochs': 10}
0.541985 (0.008503) with: {'batch_size': 10, 'epochs': 10}
0.530958 (0.011950) with: {'batch_size': 10, 'epochs': 50}
0.529262 (0.004760) with: {'batch_size': 10, 'epochs': 100}
0.504241 (0.016567) with: {'batch_size': 10, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 20, 'epochs': 10}
0.535623 (0.002748) with: {'batch_size': 20, 'epochs': 50}
0.531807 (0.007197) with: {'batch_size': 20, 'epochs': 100}
0.498728 (0.013584) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.536472 (0.003174) with: {'batch_size': 40, 'epochs': 50}
0.535623 (0.008113) with: {'batch_size': 40, 'epochs': 100}
0.519932 (0.008956) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536896 (0.002078) with: {'batch_size': 60, 'epochs': 50}
0.535199 (0.009873) with: {'batch_size': 60, 'epochs': 100}
0.525021 (0.015314) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.533927 (0.003174) with: {'batch_size': 80, 'epochs': 50}
0.536896 (0.003745) with: {'batch_size': 80, 'epochs': 100}
0.519508 (0.004684) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.540288 (0.003339) with: {'batch_size': 100, 'epochs': 50}
0.535199 (0.002399) with: {'batch_size': 100, 'epochs': 100}
0.532231 (0.014142) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 80, 'epochs': 100}
0.543681 (0.010812) with: {'batch_size': 10, 'epochs': 10}
0.537320 (0.009818) with: {'batch_size': 10, 'epochs': 50}
0.536896 (0.005784) with: {'batch_size': 10, 'epochs': 100}
0.503393 (0.013558) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.535623 (0.005398) with: {'batch_size': 20, 'epochs': 50}
0.538592 (0.009818) with: {'batch_size': 20, 'epochs': 100}
0.498728 (0.011236) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.535199 (0.003933) with: {'batch_size': 40, 'epochs': 50}
0.541561 (0.015970) with: {'batch_size': 40, 'epochs': 100}
0.518660 (0.014593) with: {'batch_size': 40, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 60, 'epochs': 10}
0.531807 (0.001799) with: {'batch_size': 60, 'epochs': 50}
0.538592 (0.013558) with: {'batch_size': 60, 'epochs': 100}
0.518236 (0.010249) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.534351 (0.001799) with: {'batch_size': 80, 'epochs': 50}
0.544105 (0.013950) with: {'batch_size': 80, 'epochs': 100}
0.516964 (0.006597) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.540288 (0.004909) with: {'batch_size': 100, 'epochs': 50}
0.529686 (0.001587) with: {'batch_size': 100, 'epochs': 100}
0.529686 (0.011443) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 100, 'epochs': 10}
0.541561 (0.008650) with: {'batch_size': 10, 'epochs': 10}
0.527566 (0.006087) with: {'batch_size': 10, 'epochs': 50}
0.533503 (0.012480) with: {'batch_size': 10, 'epochs': 100}
0.505937 (0.012988) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 20, 'epochs': 10}
0.534775 (0.004198) with: {'batch_size': 20, 'epochs': 50}
0.524597 (0.006597) with: {'batch_size': 20, 'epochs': 100}
0.509330 (0.006087) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 40, 'epochs': 10}
0.539864 (0.004798) with: {'batch_size': 40, 'epochs': 50}
0.532231 (0.006087) with: {'batch_size': 40, 'epochs': 100}
0.516539 (0.020854) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.537744 (0.003339) with: {'batch_size': 60, 'epochs': 50}
0.540712 (0.016718) with: {'batch_size': 60, 'epochs': 100}
0.527142 (0.014180) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.536472 (0.001587) with: {'batch_size': 80, 'epochs': 50}
0.543257 (0.004528) with: {'batch_size': 80, 'epochs': 100}
0.527566 (0.009873) with: {'batch_size': 80, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 100, 'epochs': 10}
0.534351 (0.001039) with: {'batch_size': 100, 'epochs': 50}
0.536047 (0.008835) with: {'batch_size': 100, 'epochs': 100}
0.523749 (0.014027) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 20, 'epochs': 10}
0.544529 (0.011236) with: {'batch_size': 10, 'epochs': 10}
0.532231 (0.011950) with: {'batch_size': 10, 'epochs': 50}
0.538592 (0.019609) with: {'batch_size': 10, 'epochs': 100}
0.516115 (0.017882) with: {'batch_size': 10, 'epochs': 500}
0.544529 (0.011980) with: {'batch_size': 20, 'epochs': 10}
0.531807 (0.007272) with: {'batch_size': 20, 'epochs': 50}
0.530110 (0.008712) with: {'batch_size': 20, 'epochs': 100}
0.513147 (0.008461) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.539864 (0.005229) with: {'batch_size': 40, 'epochs': 50}
0.539440 (0.010543) with: {'batch_size': 40, 'epochs': 100}
0.508482 (0.002999) with: {'batch_size': 40, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 60, 'epochs': 10}
0.539440 (0.006487) with: {'batch_size': 60, 'epochs': 50}
0.529262 (0.010543) with: {'batch_size': 60, 'epochs': 100}
0.521204 (0.015766) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.533079 (0.001799) with: {'batch_size': 80, 'epochs': 50}
0.531383 (0.012694) with: {'batch_size': 80, 'epochs': 100}
0.517388 (0.018241) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.536896 (0.004760) with: {'batch_size': 100, 'epochs': 50}
0.539016 (0.008397) with: {'batch_size': 100, 'epochs': 100}
0.519508 (0.016104) with: {'batch_size': 100, 'epochs': 500}
Best: 0.543681 using {'batch_size': 10, 'epochs': 10}
0.543681 (0.010812) with: {'batch_size': 10, 'epochs': 10}
0.537320 (0.008835) with: {'batch_size': 10, 'epochs': 50}
0.536047 (0.014849) with: {'batch_size': 10, 'epochs': 100}
0.499152 (0.026532) with: {'batch_size': 10, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 20, 'epochs': 10}
0.538168 (0.008310) with: {'batch_size': 20, 'epochs': 50}
0.538168 (0.015720) with: {'batch_size': 20, 'epochs': 100}
0.505089 (0.008245) with: {'batch_size': 20, 'epochs': 500}
0.543257 (0.010994) with: {'batch_size': 40, 'epochs': 10}
0.536896 (0.001799) with: {'batch_size': 40, 'epochs': 50}
0.540288 (0.009426) with: {'batch_size': 40, 'epochs': 100}
0.520780 (0.003339) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.536472 (0.004325) with: {'batch_size': 60, 'epochs': 50}
0.537320 (0.013235) with: {'batch_size': 60, 'epochs': 100}
0.522477 (0.020179) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.538168 (0.001799) with: {'batch_size': 80, 'epochs': 50}
0.528838 (0.008461) with: {'batch_size': 80, 'epochs': 100}
0.528838 (0.004684) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.537744 (0.004325) with: {'batch_size': 100, 'epochs': 50}
0.539016 (0.003174) with: {'batch_size': 100, 'epochs': 100}
0.520356 (0.015089) with: {'batch_size': 100, 'epochs': 500}
Best: 0.543681 using {'batch_size': 40, 'epochs': 10}
0.541561 (0.007934) with: {'batch_size': 10, 'epochs': 10}
0.535199 (0.014556) with: {'batch_size': 10, 'epochs': 50}
0.539440 (0.009056) with: {'batch_size': 10, 'epochs': 100}
0.504665 (0.014740) with: {'batch_size': 10, 'epochs': 500}
0.542833 (0.009652) with: {'batch_size': 20, 'epochs': 10}
0.529686 (0.003933) with: {'batch_size': 20, 'epochs': 50}
0.540712 (0.006812) with: {'batch_size': 20, 'epochs': 100}
0.504241 (0.004198) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.533079 (0.002078) with: {'batch_size': 40, 'epochs': 50}
0.535199 (0.012566) with: {'batch_size': 40, 'epochs': 100}
0.512723 (0.023960) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.533079 (0.002748) with: {'batch_size': 60, 'epochs': 50}
0.533079 (0.003745) with: {'batch_size': 60, 'epochs': 100}
0.515267 (0.018023) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.537320 (0.007071) with: {'batch_size': 80, 'epochs': 50}
0.538168 (0.006812) with: {'batch_size': 80, 'epochs': 100}
0.513571 (0.021946) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.532231 (0.004684) with: {'batch_size': 100, 'epochs': 50}
0.539440 (0.004155) with: {'batch_size': 100, 'epochs': 100}
0.531383 (0.013397) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544105 using {'batch_size': 40, 'epochs': 10}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.542833 (0.000600) with: {'batch_size': 10, 'epochs': 50}
0.533079 (0.007491) with: {'batch_size': 10, 'epochs': 100}
0.508058 (0.013950) with: {'batch_size': 10, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 20, 'epochs': 10}
0.533503 (0.002162) with: {'batch_size': 20, 'epochs': 50}
0.529686 (0.004909) with: {'batch_size': 20, 'epochs': 100}
0.516539 (0.016718) with: {'batch_size': 20, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 40, 'epochs': 10}
0.540712 (0.000000) with: {'batch_size': 40, 'epochs': 50}
0.537744 (0.007797) with: {'batch_size': 40, 'epochs': 100}
0.524173 (0.010018) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535623 (0.002748) with: {'batch_size': 60, 'epochs': 50}
0.535623 (0.015089) with: {'batch_size': 60, 'epochs': 100}
0.522053 (0.004198) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.530958 (0.004909) with: {'batch_size': 80, 'epochs': 50}
0.538168 (0.003745) with: {'batch_size': 80, 'epochs': 100}
0.526293 (0.015628) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.537744 (0.000600) with: {'batch_size': 100, 'epochs': 50}
0.531383 (0.006597) with: {'batch_size': 100, 'epochs': 100}
0.528414 (0.010405) with: {'batch_size': 100, 'epochs': 500}
Best: 0.544529 using {'batch_size': 20, 'epochs': 10}
0.543257 (0.010231) with: {'batch_size': 10, 'epochs': 10}
0.535623 (0.007491) with: {'batch_size': 10, 'epochs': 50}
0.537744 (0.008461) with: {'batch_size': 10, 'epochs': 100}
0.505089 (0.008113) with: {'batch_size': 10, 'epochs': 500}
0.544529 (0.010543) with: {'batch_size': 20, 'epochs': 10}
0.536047 (0.001587) with: {'batch_size': 20, 'epochs': 50}
0.530958 (0.007515) with: {'batch_size': 20, 'epochs': 100}
0.511874 (0.016171) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.533927 (0.007657) with: {'batch_size': 40, 'epochs': 50}
0.530110 (0.014849) with: {'batch_size': 40, 'epochs': 100}
0.513995 (0.006319) with: {'batch_size': 40, 'epochs': 500}
0.544105 (0.011395) with: {'batch_size': 60, 'epochs': 10}
0.527990 (0.007272) with: {'batch_size': 60, 'epochs': 50}
0.541137 (0.009873) with: {'batch_size': 60, 'epochs': 100}
0.510602 (0.015731) with: {'batch_size': 60, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 80, 'epochs': 10}
0.538168 (0.003599) with: {'batch_size': 80, 'epochs': 50}
0.534775 (0.010611) with: {'batch_size': 80, 'epochs': 100}
0.512299 (0.003933) with: {'batch_size': 80, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 100, 'epochs': 10}
0.537744 (0.003174) with: {'batch_size': 100, 'epochs': 50}
0.529686 (0.009426) with: {'batch_size': 100, 'epochs': 100}
0.525445 (0.006812) with: {'batch_size': 100, 'epochs': 500}

Best: 0.545377 using {'batch_size': 20, 'epochs': 100}
0.542833 (0.008956) with: {'batch_size': 10, 'epochs': 10}
0.538168 (0.016326) with: {'batch_size': 10, 'epochs': 50}
0.538592 (0.011814) with: {'batch_size': 10, 'epochs': 100}
0.494911 (0.007491) with: {'batch_size': 10, 'epochs': 500}
0.544105 (0.009982) with: {'batch_size': 20, 'epochs': 10}
0.536896 (0.003116) with: {'batch_size': 20, 'epochs': 50}
0.545377 (0.006597) with: {'batch_size': 20, 'epochs': 100}
0.510602 (0.005229) with: {'batch_size': 20, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 40, 'epochs': 10}
0.532655 (0.005124) with: {'batch_size': 40, 'epochs': 50}
0.542833 (0.005229) with: {'batch_size': 40, 'epochs': 100}
0.513147 (0.017516) with: {'batch_size': 40, 'epochs': 500}
0.543681 (0.010812) with: {'batch_size': 60, 'epochs': 10}
0.535623 (0.003745) with: {'batch_size': 60, 'epochs': 50}
0.541561 (0.009652) with: {'batch_size': 60, 'epochs': 100}
0.526718 (0.015582) with: {'batch_size': 60, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 80, 'epochs': 10}
0.539864 (0.006917) with: {'batch_size': 80, 'epochs': 50}
0.540712 (0.002748) with: {'batch_size': 80, 'epochs': 100}
0.519084 (0.017006) with: {'batch_size': 80, 'epochs': 500}
0.543257 (0.010231) with: {'batch_size': 100, 'epochs': 10}
0.536047 (0.005331) with: {'batch_size': 100, 'epochs': 50}
0.540712 (0.004528) with: {'batch_size': 100, 'epochs': 100}
0.533927 (0.015628) with: {'batch_size': 100, 'epochs': 500}